In [1]:
import PublicDataReader as pdr
from config import OpenAPI
import pandas as pd

# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

print(pdr.__all__)
print(pdr.__version__)
print(pdr.__info__)

['__version__', '__info__', 'Transaction', 'Building', 'code_list']
2021.11.
- Author : Wooil Jeong
- E-mail : wooil@kakao.com
- Github : https://github.com/WooilJeong/PublicDataReader
- Blog : https://wooiljeong.github.io


In [2]:
# 법정동코드 목록
code = pdr.code_list()
code.head()

,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,NaN,1100000000,NaN,NaN,19880423,NaN
1,11,서울특별시,11110,종로구,1111000000,NaN,NaN,19880423,NaN
2,11,서울특별시,11110,종로구,1111010100,청운동,NaN,19880423,NaN
3,11,서울특별시,11110,종로구,1111010200,신교동,NaN,19880423,NaN
4,11,서울특별시,11110,종로구,1111010300,궁정동,NaN,19880423,NaN


In [12]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Transaction(serviceKey, debug=False)

In [13]:
prod="분양입주권"
trans="매매"
sigunguCode="41135"
yearMonth="202101"

df = self.read_data(prod, trans, sigunguCode, yearMonth)
df

,지역코드,시군구,법정동,지번,단지,층,전용면적,구분,년,월,일,거래금액,거래유형,중개사소재지,해제사유발생일,해제여부
0,41135,성남분당구,정자동,215,분당 더샵 파크리버,15,84.9987,,2021,1,18,"104,535",,,,


In [15]:
df = self.collect_data(prod, trans, sigunguCode, "201901", "202111")
df

,지역코드,시군구,법정동,지번,단지,층,전용면적,구분,년,월,일,거래금액,거래유형,중개사소재지,해제사유발생일,해제여부
0,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,1,84.8034,,2019,1,22,"77,570",,,,
1,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,21,84.9746,,2019,1,22,"94,580",,,,
2,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,17,129.9777,,2019,1,22,"132,430",,,,
3,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,12,84.9746,,2019,1,22,"92,940",,,,
4,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,5,84.9746,,2019,1,22,"90,770",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,24,129.8347,,2021,4,24,"214,000",,,,
270,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,2,114.8623,,2021,4,25,"173,000",,,,
271,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,13,114.8623,,2021,4,26,"171,970",,,,
272,41135,성남분당구,백현동,516,판교 더샵 퍼스트파크,5,84.8034,,2021,5,28,"141,000",,,,


## Building Class

## Transaction Class

In [ ]:
# 11종 데이터 조회 테스트
sigunguCode = "11680"
yearMonth = "202110"

df_list = []
for prod in self.metaDict.keys():
    for trans in self.metaDict[prod].keys():
        print()
        print("▶", prod, trans)
        print()
        df= self.read_data(prod, trans, sigunguCode, yearMonth)
        df_list.append(df)
        print(df.head(1))

In [ ]:
prod = "아파트"
trans = "매매"
sigunguCode = "11680"
yearMonth = "202110"
startYearMonth = "202101"
endYearMonth = "202103"

In [ ]:
df = self.read_data(prod, trans, sigunguCode, yearMonth)
df

In [ ]:
df_sum = self.collect_data(prod, trans, sigunguCode, startYearMonth, endYearMonth)
df_sum

## 컬럼 리스트 추출

In [ ]:
# LAWD_CD=11680
# DEAL_YMD=202110


# for prod in ['아파트','오피스텔','단독다가구','연립다세대','상업업무용','분양입주권','토지']:
#     for trans in ['매매', '전월세']:
#         try:
#             print()
#             print(f">> {prod} {trans}")
#             print()
#             service_url = self.metaDict[prod][trans]['url']

#             # URL
#             url=f"""\
# {service_url}\
# &LAWD_CD={str(LAWD_CD)}\
# &DEAL_YMD={str(DEAL_YMD)}\
# &numOfRows=99999\
#             """

#             result = requests.get(url, verify=False)
#             xmlsoup = BeautifulSoup(result.text, "lxml-xml")
#             item = xmlsoup.find("item")

#             col_list = []
#             for i in item:
#                 col_list.append(i.name)
#             for col in col_list: 
#                 print(f"'{col}'", end=',')
                
#         except:
#             print("Error")
#             pass

### Features

- 공통

    - 로깅 기능
        - 세션 초기화 시 `debug` 인자 추가
            - False(Default): 오류 레벨만 출력
            - True: 모든 레벨 출력
        - OPEN API 에러 코드 출력

    - Air Korea Open API 모듈 제거
    
    - 법정동코드 테이블 조회 함수 추가

- 부동산 실거래가

    - Operation 별 조회 항목 수정
        - 매매자료 '해제사유발생일' 항목 추가
        - 거래유형, 중개사소재지 항목 추가
    - 데이터 조회 메서드 일원화
        - molit.read_data('아파트', '매매', '11680', '202111')
            - 서비스별 End-point 및 컬럼 매핑 딕셔너리 추가
    - 데이터 수집 메서드 신규 메서드 추가
        - molit.collect_data('아파트', '매매', '11680', '202101', '202103')

- 건축물대장

